In [ ]:
%load_ext autoreload
%autoreload 2

import resource

# constrain our own RAM for safety
resource.setrlimit(resource.RLIMIT_AS, (4 * 1024 * 1024 * 1024, resource.RLIM_INFINITY))

In [ ]:
import os
from pprint import pp

from evo.data_converters.common import create_evo_object_service_and_data_client
from evo.notebooks import ServiceManagerWidget

client_id = os.getenv("EVO_CLIENT_ID", "")
base_uri = os.getenv("EVO_BASE_URI", "")
discovery_url = os.getenv("EVO_DISCOVERY_URL", "")

manager = await ServiceManagerWidget.with_auth_code(
    client_id=client_id, base_uri=base_uri, discovery_url=discovery_url
).login()

object_service_client, data_client = create_evo_object_service_and_data_client(service_manager_widget=manager)

In [ ]:
import ipywidgets as widgets

from pathlib import Path

data_dir = Path.cwd() / "data"
obj_files = [p.relative_to(data_dir) for p in data_dir.rglob("*.obj")]

file_selector = widgets.RadioButtons(options=obj_files, disabled=False)

display(file_selector)

In [ ]:
from evo.data_converters.obj.importer.obj_to_evo import convert_obj

test_file = data_dir / file_selector.value
assert test_file.exists()

tags = {"TagName": "Tag value"}
implementation = "trimesh"
objects_metadata = convert_obj(
    filepath=test_file,
    implementation=implementation,
    epsg_code=32650,
    service_manager_widget=manager,
    tags=tags,
    upload_path=f"test-obj-{implementation}",
    publish_objects=False,
    overwrite_existing_objects=True,
)

print()
print("These objects have now been published:")

for metadata in objects_metadata:
    pp(metadata, indent=4)